# Estimate Metalinks

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import muon as mu
import liana as li

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from liana.funcomics import neg_to_zero

In [3]:
import decoupler as dc

Test object

In [4]:
adata = sc.datasets.pbmc68k_reduced()
adata = adata.raw.to_adata()

## Metabolite Enrichment

In [5]:
resource = pd.read_csv("liana/resource/metalinks_resource.csv")

In [6]:
resource = resource[['HMDB', 'Symbol']].rename(columns={'HMDB': 'ligand', 'Symbol': 'receptor'})

In [7]:
resource

,ligand,receptor
0,HMDB0000112,GABRB2
1,HMDB0001043,CLCN2
2,HMDB0015180,CLCN2
3,HMDB0000216,ADRA2B
4,HMDB0000068,ADRA2B
...,...,...
10160,HMDB0041130,MUSK
10161,HMDB0002496,MUSK
10162,HMDB0015397,MUSK
10163,HMDB0006284,MUSK


Process net

In [8]:
# replace direction with 1 or -1
met_net = pd.read_csv("liana/resource/metabolite_sets.csv")
met_net['direction'].replace({'degrading': -1, 'producing': 1}, inplace=True)

In [9]:
# NOTE: there are duplicates, Elias should append Sources
met_net = met_net.drop_duplicates(['HMDB', 'GENE', 'direction'])

In [10]:
# NOTE: there are sign-duplicates...
met_net = met_net.groupby(['HMDB', 'GENE']).aggregate({'direction': 'mean'}).reset_index()
# Remove any edge whose direction is not 1 or -1
met_net = met_net[met_net['direction'].isin([-1, 1])]

Metabolite Estimate

In [11]:
dc.run_ulm(adata, net=met_net, source='HMDB', target='GENE', weight='direction', use_raw=False, min_n=2)

In [12]:
met_est = li.fun.obsm_to_adata(adata, 'ulm_estimate')
met_est

AnnData object with n_obs × n_vars = 700 × 95
    obs: 'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    uns: 'bulk_labels_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'ulm_estimate', 'ulm_pvals'
    obsp: 'distances', 'connectivities'

To MuData

In [13]:
mdata = mu.MuData({'metabolite':met_est, 'rna':adata})

Run LIANA

In [14]:
li.mt.rank_aggregate(mdata, groupby='bulk_labels',
                     multi_kwargs=dict(x_mod='metabolite', y_mod='rna',
                                       x_transform = neg_to_zero,
                                       y_transform = False),
                     n_perms=None, use_raw=False, verbose=True,
                     resource=resource)

Using `.X`!
Converting mat to CSR format
Transforming metabolite using neg_to_zero
Using `.X`!
Using `.X`!
Using provided `resource`.
0.98 of entities in the resource are missing from the data.
Generating ligand-receptor stats for 700 samples and 860 features
Assuming that counts were `natural` log-normalized!
Running CellPhoneDB
Running Connectome
Running log2FC
Running NATMI
Running SingleCellSignalR
Running CellChat


In [15]:
mdata.uns['liana_res']

,source,target,ligand_complex,receptor_complex,lr_means,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,lr_probs,magnitude_rank
12,CD4+/CD45RO+ Memory,CD4+/CD45RA+/CD25- Naive T,HMDB0000123,CXCR4,0.768593,0.273871,0.883349,0.745058,0.068262,0.479293,0.004321,0.000036
15,CD14+ Monocyte,CD4+/CD45RA+/CD25- Naive T,HMDB0000123,CXCR4,0.721617,0.148773,0.663560,0.628908,0.037081,0.404200,0.001795,0.005583
42,CD4+/CD45RO+ Memory,CD8+/CD45RA+ Naive Cytotoxic,HMDB0000123,CXCR4,0.589145,0.200051,0.648027,0.519143,0.049862,0.440306,0.003105,0.020353
83,CD56+ NK,CD14+ Monocyte,HMDB0001341,P2RY13,0.697882,0.237026,0.682175,0.731639,0.099824,0.461297,0.000000,0.023357
45,CD14+ Monocyte,CD8+/CD45RA+ Naive Cytotoxic,HMDB0000123,CXCR4,0.542169,0.108673,0.428237,0.402993,0.027086,0.367017,0.001289,0.042129
...,...,...,...,...,...,...,...,...,...,...,...,...
96,CD19+ B,CD19+ B,HMDB0000123,CXCR4,0.133630,0.009516,-0.224411,-0.406387,0.002372,0.146451,0.000000,1.000000
113,CD8+ Cytotoxic T,CD56+ NK,HMDB0000123,CXCR4,0.243890,0.026961,-0.044037,-0.091241,0.006720,0.224087,0.000000,1.000000
114,CD8+/CD45RA+ Naive Cytotoxic,CD56+ NK,HMDB0000123,CXCR4,0.264828,0.044726,0.053929,-0.065919,0.011148,0.271125,0.000000,1.000000
34,CD8+/CD45RA+ Naive Cytotoxic,CD8+ Cytotoxic T,HMDB0000123,CXCR4,0.405512,0.074391,0.238417,0.359093,0.018542,0.324200,0.000000,1.000000


## Check Transporters

Transporter estimate

(Note worth it for this part)

In [ ]:
t_net = pd.read_csv("liana/resource/transporter_sets.csv")

In [ ]:
# when revirsible == True, else False
t_net['reversibility'] = t_net['reversibility']=='reversible'

In [ ]:
in_net = t_net[(t_net['transport_direction'] == 'in') | (t_net['reversibility'])]
in_net

In [ ]:

out_net = t_net[(t_net['transport_direction'] == 'out') | (t_net['reversibility'])]
out_net

In [ ]:
dc.run_ulm(adata, net=out_net, source='HMDB', target='GENE', weight='direction', use_raw=False, min_n=3)